In [1]:
import pandas as pd

data = pd.read_csv('./cleaned_data/final.csv', index_col=0)
data.head()


,path_failure,path_amount,hop_0_destination_pubkey,hop_0_source_pubkey,hop_0_is_final_hop,hop_0_scid,hop_1_destination_pubkey,hop_1_source_pubkey,hop_1_is_final_hop,hop_1_scid,...,hop_10_is_final_hop,hop_10_scid,hop_11_destination_pubkey,hop_11_source_pubkey,hop_11_is_final_hop,hop_11_scid,hop_12_destination_pubkey,hop_12_source_pubkey,hop_12_is_final_hop,hop_12_scid
0,1,50000000,2,0,0,1,1,2,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,50000000,3,0,0,2,4,3,0,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1,50000000,2,0,0,1,6,2,0,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,50000000,3,0,0,2,7,3,0,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,50000000,3,0,0,2,8,3,0,10,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [2]:
# Read the data back from the CSV files
features_train = pd.read_csv('cleaned_data/features_train.csv', index_col=0)
target_train = pd.read_csv('cleaned_data/target_train.csv', index_col=0)
features_test = pd.read_csv('cleaned_data/features_test.csv', index_col=0)
target_test = pd.read_csv('cleaned_data/target_test.csv', index_col=0)
features_val = pd.read_csv('cleaned_data/features_val.csv', index_col=0)
target_val = pd.read_csv('cleaned_data/target_val.csv', index_col=0)


In [7]:
import torch
from torch import nn
import pytorch_lightning as pl

class FeedForwardNN(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.layer3 = nn.Linear(32, output_dim)
        self.loss = nn.BCEWithLogitsLoss()
        self.val_losses = []

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.layer3(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        self.log('val_loss', loss)
        self.val_losses.append(loss)

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.val_losses).mean()
        self.log('avg_val_loss', avg_loss)
        self.val_losses = []  # reset for the next epoch

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [8]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
features_train_tensor = torch.tensor(features_train.values, dtype=torch.float32)
target_train_tensor = torch.tensor(target_train.values, dtype=torch.float32)

# Create a DataLoader
train_data = TensorDataset(features_train_tensor, target_train_tensor)
train_loader = DataLoader(train_data, batch_size=32)

# Convert validation data to PyTorch tensors
features_val_tensor = torch.tensor(features_val.values, dtype=torch.float32)
target_val_tensor = torch.tensor(target_val.values, dtype=torch.float32)

# Create a DataLoader for the validation data
val_data = TensorDataset(features_val_tensor, target_val_tensor)
val_loader = DataLoader(val_data, batch_size=32)

# Train the model
model = FeedForwardNN(features_train.shape[1], 1)
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type              | Params
---------------------------------------------
0 | layer1 | Sequential        | 3.6 K 
1 | layer2 | Sequential        | 2.1 K 
2 | layer3 | Linear            | 33    
3 | loss   | BCEWithLogitsLoss | 0     
---------------------------------------------
5.8 K     Trainable params
0         Non-trainable params
5.8 K     Total params
0.023     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [9]:
torch.save(model.state_dict(), './saved_models/ffnn_model.pth')
